# Code Carbon Tracking
This notebooks is an example of how we can track the carbon impact of our work within the AI Lab.

We have several methods dependent on what we want to track.

We use a Python Package called CodeCarbon (https://github.com/mlco2/codecarbon) for this.

### Tracking whole Notebook
To track the whole notebook we need to start the tracker at the beginning of the notebook, and stop the tracker in the end of the notebook.

We can also use this method for tracking 

In [1]:
# Codeblock where we import the EmissionsTracker and start the tracker.
# This should be placed at the beginning of the notebook if we want to track the whole notebook.
from codecarbon import EmissionsTracker
tracker = EmissionsTracker()
tracker.start()

[codecarbon INFO @ 12:58:33] [setup] RAM Tracking...
[codecarbon INFO @ 12:58:33] [setup] GPU Tracking...
[codecarbon INFO @ 12:58:34] No GPU found.
[codecarbon INFO @ 12:58:34] [setup] CPU Tracking...
[codecarbon WARNING @ 12:58:34] No CPU tracking mode found. Falling back on CPU constant mode. 
 Linux OS detected: Please ensure RAPL files exist at \sys\class\powercap\intel-rapl to measure CPU

[codecarbon INFO @ 12:58:35] CPU Model on constant consumption mode: Intel(R) Xeon(R) Platinum 8171M CPU @ 2.60GHz
[codecarbon INFO @ 12:58:35] >>> Tracker's metadata:
[codecarbon INFO @ 12:58:35]   Platform system: Linux-5.15.0-1064-azure-x86_64-with-glibc2.31
[codecarbon INFO @ 12:58:35]   Python version: 3.10.14
[codecarbon INFO @ 12:58:35]   CodeCarbon version: 2.7.1
[codecarbon INFO @ 12:58:35]   Available RAM : 13.648 GB
[codecarbon INFO @ 12:58:35]   CPU count: 2
[codecarbon INFO @ 12:58:35]   CPU model: Intel(R) Xeon(R) Platinum 8171M CPU @ 2.60GHz
[codecarbon INFO @ 12:58:35]   GPU cou

In [2]:
# Codeblock where we stop the tracker.
# This should be placed at the end of the notebook if we want to track the whole notebook.
tracker.stop()

[codecarbon INFO @ 12:58:50] Energy consumed for RAM : 0.000021 kWh. RAM Power : 5.1181769371032715 W
[codecarbon INFO @ 12:58:50] Energy consumed for all CPUs : 0.000336 kWh. Total CPU Power : 82.5 W
[codecarbon INFO @ 12:58:50] 0.000357 kWh of electricity used since the beginning.
/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/codecarbon/output_methods/file.py:52: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame.from_records([dict(total.values)])])


9.551571991815342e-05

### Tracking multiple _small_ pieces of code individually
To track a small piece of code we use the _task manager_. 

This way CodeCarbon will track the emissions of each task. 

The task will not be written to disk to prevent overhead, you have to get the results from the return of `stop_task()`. 

If no name is provided, CodeCarbon will generate a uuid.



In [3]:
from codecarbon import EmissionsTracker
import datasets

try:
    tracker = EmissionsTracker(project_name="small_pieces", measure_power_secs=10)
    # We start a specific task using the tracker
    tracker.start_task("load dataset")
    dataset = load_dataset("imdb", split="test")
    # We have to stop individual tasks
    imdb_emissions = tracker.stop_task()
    tracker.start_task("build model")
    model = build_model()
    model_emissions = tracker.stop_task()
finally:
    # We 
    _ = tracker.stop()

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[codecarbon INFO @ 12:58:59] [setup] RAM Tracking...
[codecarbon INFO @ 12:58:59] [setup] GPU Tracking...
[codecarbon INFO @ 12:59:00] No GPU found.
[codecarbon INFO @ 12:59:00] [setup] CPU Tracking...
[codecarbon WARNING @ 12:59:00] No CPU tracking mode found. Falling back on CPU constant mode. 
 Linux OS detected: Please ensure RAPL files exist at \sys\class\powercap\intel-rapl to measure CPU

[codecarbon INFO @ 12:59:01] CPU Model on constant consumption mode: Intel(R) Xeon(R) Platinum 8171M CPU @ 2.60GHz
[codecarbon INFO @ 12:59:01] >>> Tracker's metadata:
[codecarbon INFO @ 12:59:01]   Platform system: Linux-5.15.0-1064-azure-x86_64-with-glibc2.31
[codecarbon INFO @ 12:59:01]   Python version: 3.10.14
[codecarbon INF

NameError: name 'load_dataset' is not defined

Please note that you can’t use task mode and normal mode at the same time. 

Because `start_task` will stop the scheduler to prevent it interfering with the task measurement.



### Tracking a specific codeblock
To track a specific codeblock we can use the `EmissionsTracker` as a context manager.

In [ ]:
from codecarbon import EmissionsTracker

with EmissionsTracker() as tracker:
    # Compute intensive training code goes here

### Tracking a function
To track code wrapped within a function, we can use the decorator `@track_emissions` within the function.

In [ ]:
from codecarbon import track_emissions

@track_emissions
def training_loop():
    # Compute intensive training code goes here

## Examples
Following are examples to train a Deep Learning model on MNIST Data to recognize digits in images using TensorFlow. 

### Using the Explicit Object
We can use this to simply start and stop a tracker object and track all code between start and stop.

In [4]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import time

from codecarbon import EmissionsTracker

# We create some random data to 
np.random.seed(42)  
X = np.random.rand(1000, 5)  
y = (np.sum(X, axis=1) > 2.5).astype(int) 

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train a simple logistic regression model
model = LogisticRegression()

# Initialize and start the emissions tracker
tracker = EmissionsTracker('explicit_object')
tracker.start()

# Fit the model
model.fit(X_train, y_train)

# We sleep for 5 sec to actually have some results
time.sleep(5)

# Stop the emissions tracker and output the emissions data
emissions: float =  tracker.stop()
print(f"Emissions during training: {emissions:.8f} kg CO2eq")


[codecarbon INFO @ 12:59:13] [setup] RAM Tracking...
[codecarbon INFO @ 12:59:13] [setup] GPU Tracking...
[codecarbon INFO @ 12:59:15] No GPU found.
[codecarbon INFO @ 12:59:15] [setup] CPU Tracking...
[codecarbon WARNING @ 12:59:15] No CPU tracking mode found. Falling back on CPU constant mode. 
 Linux OS detected: Please ensure RAPL files exist at \sys\class\powercap\intel-rapl to measure CPU

[codecarbon INFO @ 12:59:16] CPU Model on constant consumption mode: Intel(R) Xeon(R) Platinum 8171M CPU @ 2.60GHz
[codecarbon INFO @ 12:59:16] >>> Tracker's metadata:
[codecarbon INFO @ 12:59:16]   Platform system: Linux-5.15.0-1064-azure-x86_64-with-glibc2.31
[codecarbon INFO @ 12:59:16]   Python version: 3.10.14
[codecarbon INFO @ 12:59:16]   CodeCarbon version: 2.7.1
[codecarbon INFO @ 12:59:16]   Available RAM : 13.648 GB
[codecarbon INFO @ 12:59:16]   CPU count: 2
[codecarbon INFO @ 12:59:16]   CPU model: Intel(R) Xeon(R) Platinum 8171M CPU @ 2.60GHz
[codecarbon INFO @ 12:59:16]   GPU cou

Emissions during training: 0.00003267 kg CO2eq


### Using the Context Manager

In [5]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import time

from codecarbon import EmissionsTracker

# We create some random data to 
np.random.seed(42)  
X = np.random.rand(1000, 5)  
y = (np.sum(X, axis=1) > 2.5).astype(int) 

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train a simple logistic regression model
model = LogisticRegression()


with EmissionsTracker('context_manager') as tracker:
    # Fit the model
    model.fit(X_train, y_train)
    # We sleep for 5 sec to actually have some results
    time.sleep(5)
    

[codecarbon INFO @ 12:59:25] [setup] RAM Tracking...
[codecarbon INFO @ 12:59:25] [setup] GPU Tracking...
[codecarbon INFO @ 12:59:26] No GPU found.
[codecarbon INFO @ 12:59:26] [setup] CPU Tracking...
[codecarbon WARNING @ 12:59:26] No CPU tracking mode found. Falling back on CPU constant mode. 
 Linux OS detected: Please ensure RAPL files exist at \sys\class\powercap\intel-rapl to measure CPU

[codecarbon INFO @ 12:59:28] CPU Model on constant consumption mode: Intel(R) Xeon(R) Platinum 8171M CPU @ 2.60GHz
[codecarbon INFO @ 12:59:28] >>> Tracker's metadata:
[codecarbon INFO @ 12:59:28]   Platform system: Linux-5.15.0-1064-azure-x86_64-with-glibc2.31
[codecarbon INFO @ 12:59:28]   Python version: 3.10.14
[codecarbon INFO @ 12:59:28]   CodeCarbon version: 2.7.1
[codecarbon INFO @ 12:59:28]   Available RAM : 13.648 GB
[codecarbon INFO @ 12:59:28]   CPU count: 2
[codecarbon INFO @ 12:59:28]   CPU model: Intel(R) Xeon(R) Platinum 8171M CPU @ 2.60GHz
[codecarbon INFO @ 12:59:28]   GPU cou

### Using the Decorator

In [6]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

from codecarbon import track_emissions


@track_emissions(project_name="decorator")
def train_model():
    # We create some random data to train on
    np.random.seed(42)  
    X = np.random.rand(1000, 5)  
    y = (np.sum(X, axis=1) > 2.5).astype(int) 
    # Split the data into training and testing sets (80% train, 20% test)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Initialize and train a simple logistic regression model
    model = LogisticRegression()
    # We sleep for 5 sec to actually have some results
    time.sleep(5)
    return model

model = train_model()

[codecarbon INFO @ 12:59:44] [setup] RAM Tracking...
[codecarbon INFO @ 12:59:44] [setup] GPU Tracking...
[codecarbon INFO @ 12:59:45] No GPU found.
[codecarbon INFO @ 12:59:45] [setup] CPU Tracking...
[codecarbon WARNING @ 12:59:45] No CPU tracking mode found. Falling back on CPU constant mode. 
 Linux OS detected: Please ensure RAPL files exist at \sys\class\powercap\intel-rapl to measure CPU

[codecarbon INFO @ 12:59:46] CPU Model on constant consumption mode: Intel(R) Xeon(R) Platinum 8171M CPU @ 2.60GHz
[codecarbon INFO @ 12:59:46] >>> Tracker's metadata:
[codecarbon INFO @ 12:59:46]   Platform system: Linux-5.15.0-1064-azure-x86_64-with-glibc2.31
[codecarbon INFO @ 12:59:46]   Python version: 3.10.14
[codecarbon INFO @ 12:59:46]   CodeCarbon version: 2.7.1
[codecarbon INFO @ 12:59:46]   Available RAM : 13.648 GB
[codecarbon INFO @ 12:59:46]   CPU count: 2
[codecarbon INFO @ 12:59:46]   CPU model: Intel(R) Xeon(R) Platinum 8171M CPU @ 2.60GHz
[codecarbon INFO @ 12:59:46]   GPU cou

## Visualization
CodeCarbon comes with a `Dash App` where the emissions are visualized.

To run it, we execute the CLI command below:

`carbonboard --filepath="emissions.csv" --port=3333`

In [7]:
import os